In [ ]:
from src.llm_plan.environments.static.two_agents_blocksworld import TwoAgentsBlocksWorld

env = TwoAgentsBlocksWorld("./environments/static/TwoAgentsBlocksWorld.json")
env.plan  # Simple plan to solve the Two Agents Vault environment

In [ ]:
# Create the plan
from src.llm_plan.planner import Planner
from src.llm_plan.llm import GPT_4o

model = GPT_4o()
planner = Planner(env, model)


In [ ]:
# Plan
responses = planner.plan()

In [ ]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [ ]:
# Obtain the plan with the solver
import subprocess

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

with open("./tmp/problem.pddl", "w") as f:
    f.write(problem)

with open("./tmp/domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    "./tmp/sas_plan",
    "./tmp/domain.pddl",
    "./tmp/problem.pddl",
]

with open("./tmp/logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)